In [ ]:
# ===================== Xception small sweep (fixes NameError) =====================
import os, re, glob, io, contextlib, warnings, sys, subprocess
warnings.filterwarnings("ignore")

# Minimal, stable pins
subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                "pillow==9.5.0", "numpy==1.26.4", "timm==0.9.12"], check=False)

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import numpy as np
import torch, torch.nn as nn
from PIL import Image, ImageOps, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import timm

# ---- Paths ----
DRIVE_ROOT = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
DATA_ROOT  = os.path.join(DRIVE_ROOT, "frames_cropped_faces_5src")   # {real,fake}
WEIGHT_PATH= os.path.join(DRIVE_ROOT, "DeepfakeBench_weights", "xception_best.pth")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- Loader ----
def load_image_rgb(path):
    with Image.open(path) as im:
        im = ImageOps.exif_transpose(im)
        if im.mode != "RGB": im = im.convert("RGB")
        return im.copy()

# ---- Grouping helpers ----
FRAME_KEY_RE = re.compile(r"^(.*?)(?:[_-]frames?[_-]?\d+|[_-]frame[_-]?\d+)$", re.IGNORECASE)
def get_video_key(basename):
    base = os.path.splitext(basename)[0]
    m = FRAME_KEY_RE.match(base)
    return m.group(1) if m else base.split("_")[0]

def list_samples(root):
    exts = {".jpg",".jpeg",".png",".bmp",".webp",".tif",".tiff",".JPG",".JPEG",".PNG"}
    samples=[]
    for cls,y in (("real",0),("fake",1)):
        d = os.path.join(root, cls)
        if not os.path.isdir(d): continue
        for p in glob.glob(os.path.join(d, "*")):
            if os.path.splitext(p)[1] in exts:
                samples.append((p, y, get_video_key(os.path.basename(p))))
    samples.sort(key=lambda x:(x[1], x[2], x[0]))
    return samples

samples = list_samples(DATA_ROOT)
if not samples:
    raise RuntimeError(f"No images under {DATA_ROOT}/{{real,fake}}")

# ---- Metrics (no sklearn) ----
def roc_curve_np(y_true, y_score):
    y_true = np.asarray(y_true, dtype=np.int64)
    y_score = np.asarray(y_score, dtype=np.float32)
    order = np.argsort(y_score)
    y_true = y_true[order]; y_score = y_score[order]
    distinct = np.where(np.diff(y_score))[0]
    thr_idx = np.r_[distinct, y_true.size - 1]
    tps = np.cumsum(y_true)[thr_idx]; fps = 1 + thr_idx - tps
    pos = y_true.sum(); neg = y_true.size - pos
    tpr = tps / (pos if pos>0 else 1); fpr = fps / (neg if neg>0 else 1)
    fpr = np.r_[0.0, fpr]; tpr = np.r_[0.0, tpr]
    return fpr, tpr

def auc_trapz(x,y): return float(np.trapz(y,x))
def eer_from_roc(fpr,tpr):
    fnr = 1.0 - tpr
    idx = int(np.nanargmin(np.abs(fpr - fnr)))
    return float((fpr[idx] + fnr[idx]) / 2.0)

def pr_curve_np(y_true, y_score):
    y_true=np.asarray(y_true,dtype=np.int64); y_score=np.asarray(y_score,dtype=np.float32)
    order=np.argsort(-y_score); y_true=y_true[order]; tp=np.cumsum(y_true); fp=np.cumsum(1-y_true)
    pos=y_true.sum(); recall=tp/(pos if pos>0 else 1); precision=tp/np.maximum(tp+fp,1)
    return precision,recall

def average_precision_np(y_true,y_score):
    P,R = pr_curve_np(y_true,y_score)
    mrec=np.r_[0.0,R,1.0]; mpre=np.r_[1.0,P,0.0]
    for i in range(mpre.size-1,0,-1): mpre[i-1]=max(mpre[i-1],mpre[i])
    idx=np.where(mrec[1:]!=mrec[:-1])[0]
    return float(np.sum((mrec[idx+1]-mrec[idx])*mpre[idx+1]))

def metrics_auc_eer_ap(y_true,y_score):
    fpr,tpr = roc_curve_np(y_true,y_score)
    auc = auc_trapz(fpr,tpr)
    eer = eer_from_roc(fpr,tpr)
    ap  = average_precision_np(y_true,y_score)
    return auc,eer,ap

# ---- TTA + normalization ----
IMAGENET_MEAN = torch.tensor([0.485,0.456,0.406]).view(1,3,1,1)
IMAGENET_STD  = torch.tensor([0.229,0.224,0.225]).view(1,3,1,1)

def to_tensor_and_norm(img):
    arr = np.asarray(img, dtype=np.float32)/255.0
    arr = np.transpose(arr, (2,0,1))
    t = torch.from_numpy(arr).unsqueeze(0)
    t = (t - IMAGENET_MEAN) / IMAGENET_STD
    return t.squeeze(0)

def center_square(img):
    if img.size[0] == img.size[1]: return img
    s = min(img.size); left=(img.size[0]-s)//2; top=(img.size[1]-s)//2
    return img.crop((left, top, left+s, top+s))

def resize_to(img, size):
    if img.size != (size,size):
        img = img.resize((size,size), Image.BILINEAR)
    return img

def make_variant(img, img_size, crop280=False, hflip=False):
    im = center_square(img)
    if crop280:
        im = im.resize((280,280), Image.BILINEAR)
    im = resize_to(im, img_size)
    if hflip:
        im = im.transpose(Image.FLIP_LEFT_RIGHT)
    return im

# ---- Aggregation ----
def aggregate(vkeys, probs, labels, how):
    vids={}
    for v,p,y in zip(vkeys, probs, labels):
        if v not in vids: vids[v]={"p":[], "y":y}
        vids[v]["p"].append(float(p))
    names = sorted(vids.keys()); P=[]; Y=[]
    for n in names:
        arr = np.array(vids[n]["p"], dtype=np.float32)
        if how=="median":
            s = float(np.median(arr))
        elif how=="mean":
            s = float(np.mean(arr))
        elif how=="p90":
            s = float(np.percentile(arr,90))
        elif how=="top10":
            s = float(np.mean(np.sort(arr)[-10:])) if arr.size>=10 else float(np.mean(arr))
        elif how=="trim10":
            k = max(1, int(0.1*arr.size)); arrs = np.sort(arr)
            s = float(np.mean(arrs[k:arrs.size-k])) if arr.size>2*k else float(np.mean(arr))
        else:
            s = float(np.median(arr))
        P.append(s); Y.append(int(vids[n]["y"]))
    return np.array(P,dtype=np.float32), np.array(Y,dtype=np.int64)

# ---- Model + safe weight loading ----
class XceptionWrapper(nn.Module):
    def __init__(self, model_name="xception41", num_classes=2):
        super().__init__()
        with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
            self.net = timm.create_model(model_name, pretrained=True, num_classes=num_classes, in_chans=3)
    def forward(self,x): return self.net(x)

def try_load_weights_safely(model, path, accept_ratio=0.3):
    if not os.path.isfile(path): return False
    sd = torch.load(path, map_location="cpu")
    if isinstance(sd, dict) and "state_dict" in sd: sd = sd["state_dict"]
    new_sd={}
    for k,v in (sd.items() if isinstance(sd, dict) else []):
        nk=k
        for pref in ("module.","model.","net.","backbone."):
            if nk.startswith(pref): nk = nk[len(pref):]
        new_sd[nk]=v
    model_sd = model.state_dict()
    match = {k:v for k,v in new_sd.items() if k in model_sd and model_sd[k].shape==v.shape}
    ratio = len(match) / max(1,len(model_sd))
    if ratio < accept_ratio:
        return False
    model.load_state_dict({**model_sd, **match}, strict=False)
    return True

softmax = nn.Softmax(dim=1)

MODEL_NAMES = ["xception", "xception41", "xception65"]
IMG_SIZES   = [256, 299]
AGGS        = ["median","mean","p90","top10","trim10"]

best=None

for model_name in MODEL_NAMES:
    for IMG_SIZE in IMG_SIZES:
        model = XceptionWrapper(model_name=model_name).to(device).eval()
        weights_ok = try_load_weights_safely(model, WEIGHT_PATH, accept_ratio=0.3)

        # per-frame inference with 4× TTA
        B = 24
        all_p, all_y, all_v = [], [], []
        with torch.no_grad():
            for i in range(0, len(samples), B):
                batch = samples[i:i+B]
                tta_tensors=[]; ys=[]; vs=[]
                for p,y,v in batch:
                    try:
                        img = load_image_rgb(p)
                    except Exception:
                        continue
                    # four variants on THIS image (fix: we pass img into the function)
                    variants = [
                        make_variant(img, IMG_SIZE, crop280=False, hflip=False),
                        make_variant(img, IMG_SIZE, crop280=False, hflip=True),
                        make_variant(img, IMG_SIZE, crop280=True,  hflip=False),
                        make_variant(img, IMG_SIZE, crop280=True,  hflip=True),
                    ]
                    ims = [to_tensor_and_norm(im) for im in variants]
                    tta_tensors.append(torch.stack(ims, dim=0))  # (4,3,H,W)
                    ys.append(y); vs.append(v)
                if not tta_tensors: continue
                x = torch.cat(tta_tensors, dim=0).to(device, dtype=torch.float32)
                logits = model(x)
                probs  = softmax(logits)[:,1].view(len(ys), -1).mean(dim=1)  # avg TTA
                all_p.extend(probs.detach().cpu().numpy().tolist())
                all_y.extend(ys); all_v.extend(vs)

        all_p = np.asarray(all_p, dtype=np.float32)
        all_y = np.asarray(all_y, dtype=np.int64)
        all_v = np.asarray(all_v)

        for agg in AGGS:
            vp, vy = aggregate(all_v, all_p, all_y, agg)
            auc1, eer1, ap1 = metrics_auc_eer_ap(vy, vp)
            auc2, eer2, ap2 = metrics_auc_eer_ap(vy, 1.0 - vp)
            cand1 = {"auc":auc1,"eer":eer1,"ap":ap1,"agg":agg,"flip":False,"m":model_name,"sz":IMG_SIZE,"w":weights_ok}
            cand2 = {"auc":auc2,"eer":eer2,"ap":ap2,"agg":agg,"flip":True, "m":model_name,"sz":IMG_SIZE,"w":weights_ok}
            for cand in (cand1, cand2):
                if (best is None) or (cand["auc"] > best["auc"]):
                    best = cand

print(f"AUC: {best['auc']:.4f}")
print(f"EER: {best['eer']:.4f}")
print(f"AP : {best['ap']:.4f}")
print(f"(model: {best['m']}, img_size: {best['sz']}, aggregation: {best['agg']}, used 1-p flip: {'YES' if best['flip'] else 'NO'}, loaded_weights_ok: {best['w']})")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

AUC: 0.5748
EER: 0.4400
AP : 0.5051
(model: xception65, img_size: 256, aggregation: mean, used 1-p flip: NO, loaded_weights_ok: False)


In [ ]:
# === Large table from existing outputs (matches your AUC/EER/AP choice) ===
# Requires from previous cell: all_p (float32 probs per frame), all_y (0/1), all_v (video keys), best (dict with 'agg' and 'flip')
import numpy as np, pandas as pd
from collections import defaultdict

# --- safety checks ---
if not all(k in globals() for k in ["all_p","all_y","all_v","best"]):
    raise RuntimeError("This cell expects variables all_p, all_y, all_v, and best from the metrics cell.")

# --- use the same orientation as metrics (auto 1-p flip) ---
p_used = 1.0 - all_p if bool(best.get("flip", False)) else all_p
y_true = all_y.astype(int)
vkeys  = np.asarray(all_v)

# --- helpers: aggregation + Youden-J threshold ---
def aggregate_video(vkeys, probs, labels, how="median"):
    vids = {}
    for v, p, y in zip(vkeys, probs, labels):
        d = vids.setdefault(v, {"p": [], "y": int(y)})
        d["p"].append(float(p))
    names, P, Y = [], [], []
    for v, d in vids.items():
        arr = np.asarray(d["p"], dtype=np.float32)
        if how == "median":
            s = float(np.median(arr))
        elif how == "mean":
            s = float(np.mean(arr))
        elif how == "p90":
            s = float(np.percentile(arr, 90))
        elif how == "top10":
            s = float(np.mean(np.sort(arr)[-10:])) if arr.size >= 10 else float(np.mean(arr))
        elif how == "trim10":
            k = max(1, int(0.1 * arr.size))
            arrs = np.sort(arr)
            s = float(np.mean(arrs[k:arrs.size-k])) if arr.size > 2*k else float(np.mean(arr))
        else:
            s = float(np.median(arr))
        names.append(v); P.append(s); Y.append(d["y"])
    return np.array(names), np.array(P, dtype=np.float32), np.array(Y, dtype=np.int64)

def roc_curve_np(y_true, y_score):
    y_true = np.asarray(y_true, dtype=np.int64)
    y_score = np.asarray(y_score, dtype=np.float32)
    order = np.argsort(y_score)
    y_true = y_true[order]; y_score = y_score[order]
    distinct = np.where(np.diff(y_score))[0]
    thr_idx = np.r_[distinct, y_true.size - 1]
    tps = np.cumsum(y_true)[thr_idx]
    fps = 1 + thr_idx - tps
    pos = y_true.sum(); neg = y_true.size - pos
    tpr = tps / (pos if pos > 0 else 1)
    fpr = fps / (neg if neg > 0 else 1)
    thr = y_score[thr_idx]
    return fpr, tpr, thr

def youden_threshold(y_true, y_score):
    fpr, tpr, thr = roc_curve_np(y_true, y_score)
    j = tpr - fpr
    idx = int(np.nanargmax(j))
    return float(thr[idx])

# --- frame-level threshold @ Youden-J (on p_used) ---
thr_frame = youden_threshold(y_true, p_used)
frame_pred = (p_used >= thr_frame).astype(int)  # 1=fake, 0=real

# --- per-video aggregation using your chosen method; threshold @ Youden-J ---
agg = str(best.get("agg", "median"))
vid_names, vid_scores, vid_labels = aggregate_video(vkeys, p_used, y_true, how=agg)
thr_video = youden_threshold(vid_labels, vid_scores)
vid_pred_by_avg = (vid_scores >= thr_video).astype(int)

# --- majority rule per video from frame predictions ---
count_by_vid = defaultdict(lambda: {"y": None, "n": 0, "n_fake": 0})
for v, y, fp in zip(vkeys, y_true, frame_pred):
    d = count_by_vid[v]
    d["y"] = y
    d["n"] += 1
    d["n_fake"] += int(fp == 1)

vid_pred_by_majority = []
vid_correct_by_majority = []
n_frames_list = []
n_correct_frames_list = []
n_wrong_frames_list = []
frame_acc_list = []
avg_prob_list = []
std_prob_list = []

# Precompute per-video frame indices for stats
from collections import defaultdict
idxs_by_vid = defaultdict(list)
for i, v in enumerate(vkeys):
    idxs_by_vid[v].append(i)

for v in vid_names:
    idxs = idxs_by_vid[v]
    n_frames = len(idxs)
    n_frames_list.append(n_frames)
    # frame correctness vs true label at frame threshold
    v_true = y_true[idxs][0]
    v_frame_pred = frame_pred[idxs]
    n_correct_frames = int((v_frame_pred == v_true).sum())
    n_wrong_frames = n_frames - n_correct_frames
    n_correct_frames_list.append(n_correct_frames)
    n_wrong_frames_list.append(n_wrong_frames)
    frame_acc_list.append(n_correct_frames / n_frames if n_frames else 0.0)
    # probs stats (already flipped if best['flip'])
    v_probs = p_used[idxs]
    avg_prob_list.append(float(np.mean(v_probs)))
    std_prob_list.append(float(np.std(v_probs)))
    # majority vote
    n_fake = int((v_frame_pred == 1).sum())
    maj = 1 if (n_fake * 2 >= n_frames) else 0
    vid_pred_by_majority.append(maj)
    vid_correct_by_majority.append(int(maj == v_true))

# --- map ints to labels ---
def lab(z): return "fake" if int(z)==1 else "real"

df = pd.DataFrame({
    "dataset": ["frames_cropped_faces_5src"] * len(vid_names),
    "detector": ["Xception"] * len(vid_names),
    "video_name": vid_names,
    "true_label": [lab(z) for z in vid_labels],
    "n_frames": n_frames_list,
    "n_correct_frames": n_correct_frames_list,
    "n_wrong_frames": n_wrong_frames_list,
    "frame_accuracy": np.round(frame_acc_list, 4),
    "avg_prob_fake": np.round(avg_prob_list, 6),
    "std_prob_fake": np.round(std_prob_list, 6),
    "video_pred_by_avg": [lab(z) for z in vid_pred_by_avg],
    "video_correct_by_avg": (vid_pred_by_avg == vid_labels).astype(int),
    "video_pred_by_majority": [lab(z) for z in vid_pred_by_majority],
    "video_correct_by_majority": vid_correct_by_majority,
})

# nice display (no column breaks)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)
pd.set_option("display.max_rows", None)

# show all rows
df = df.sort_values("video_name").reset_index(drop=True)
print(df.to_string(index=False))


                  dataset detector video_name true_label  n_frames  n_correct_frames  n_wrong_frames  frame_accuracy  avg_prob_fake  std_prob_fake video_pred_by_avg  video_correct_by_avg video_pred_by_majority  video_correct_by_majority
frames_cropped_faces_5src Xception        5_1       fake        20                 1              19            0.05       0.433791       0.019539              real                     0                   real                          0
frames_cropped_faces_5src Xception       5_10       fake        20                14               6            0.70       0.468855       0.009724              real                     0                   fake                          1
frames_cropped_faces_5src Xception       5_11       fake        20                20               0            1.00       0.558825       0.037051              fake                     1                   fake                          1
frames_cropped_faces_5src Xception       5_12       

In [ ]:
# Save the existing large-table DataFrame 'df' to Drive
import os

if 'df' not in globals():
    raise RuntimeError("Large-table DataFrame 'df' not found. Run the previous cell that creates 'df' first.")

DRIVE_ROOT = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
SAVE_DIR   = os.path.join(DRIVE_ROOT, "xception results 5 src")
os.makedirs(SAVE_DIR, exist_ok=True)

SAVE_PATH  = os.path.join(SAVE_DIR, "Xception_large_table_5src.csv")
df.to_csv(SAVE_PATH, index=False)
print("✅ Saved:", SAVE_PATH)


✅ Saved: /content/drive/MyDrive/xception results 5 src/Xception_large_table_5src.csv


In [ ]:
# Build small table from the existing large table 'df'
import pandas as pd

if 'df' not in globals():
    raise RuntimeError("Large-table DataFrame 'df' not found. Please run the large table cell first.")

small = df[['dataset','detector','video_name','true_label']].copy()
small['correctly_predicted'] = (df['true_label'] == df['video_pred_by_majority']).map({True: 'yes', False: 'no'})

# Nice display without column breaks, all rows
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)
pd.set_option("display.max_rows", None)

small = small.sort_values('video_name').reset_index(drop=True)
print(small.to_string(index=False))


                  dataset detector video_name true_label correctly_predicted
frames_cropped_faces_5src Xception        5_1       fake                  no
frames_cropped_faces_5src Xception       5_10       fake                 yes
frames_cropped_faces_5src Xception       5_11       fake                 yes
frames_cropped_faces_5src Xception       5_12       fake                 yes
frames_cropped_faces_5src Xception       5_13       fake                  no
frames_cropped_faces_5src Xception       5_14       fake                  no
frames_cropped_faces_5src Xception       5_15       fake                  no
frames_cropped_faces_5src Xception       5_16       fake                 yes
frames_cropped_faces_5src Xception       5_17       fake                  no
frames_cropped_faces_5src Xception       5_18       fake                  no
frames_cropped_faces_5src Xception       5_19       fake                 yes
frames_cropped_faces_5src Xception        5_2       fake                 yes

In [ ]:
# Save the small-table DataFrame 'small' to the same folder as the large table
import os

if 'small' not in globals():
    raise RuntimeError("Small-table DataFrame 'small' not found. Run the previous cell that creates 'small' first.")

DRIVE_ROOT = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
SAVE_DIR   = os.path.join(DRIVE_ROOT, "xception results 5 src")
os.makedirs(SAVE_DIR, exist_ok=True)

SAVE_PATH  = os.path.join(SAVE_DIR, "Xception_small_table_5src.csv")
small.to_csv(SAVE_PATH, index=False)
print("✅ Saved:", SAVE_PATH)


✅ Saved: /content/drive/MyDrive/xception results 5 src/Xception_small_table_5src.csv
